In [99]:
!pip install langchain_core
!pip install langchain
!pip install langchain_community
!pip install arxiv
!pip install pypdf
!pip install faiss-gpu
!pip install groq
!pip install fastembed
!pip install wikipedia
!pip install python-dotenv
!pip install langchain-groq
!pip install sentence-transformers

In [100]:
!pip install pypdf
!pip install fastembed

In [101]:
import os
from dotenv import load_dotenv
load_dotenv()

False

In [102]:
from langchain_groq import ChatGroq
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

Taking Content from Wikipedia

In [103]:
api_wrapper = WikipediaAPIWrapper(top_k_results = 2 , max_content_chars_max = 200)
wiki= WikipediaQueryRun(api_wrapper=api_wrapper)

In [104]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from fastembed import TextEmbedding
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import DirectoryLoader

In [105]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
api = os.getenv("GROQ_API_KEY")
Model = "llama-3.1-70b-versatile"
llm = ChatGroq(api_key=api,model=Model)

In [106]:
def load_pdf(data):
  loader = DirectoryLoader(data,glob="*.pdf",loader_cls = PyPDFLoader)
  documents = loader.load()
  return documents

extracted_data = load_pdf("data/")

In [107]:
def text_split(extracted_data):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap=200)
  text_chunks = text_splitter.split_documents(extracted_data)
  return text_chunks

In [108]:
text_chunks = text_split(extracted_data)
print(f"Length of the Chunk {len(text_chunks)}")

Length of the Chunk 4445


In [109]:
def embedding_model():
  model_name = "sentence-transformers/all-mpnet-base-v2"
  embedding = HuggingFaceEmbeddings(model_name = model_name)
  return embedding

In [110]:
embeddings = embedding_model()

In [111]:
sample_embedding = embeddings.embed_query("CDK")
print(f"Length of the sample_embedding{len(sample_embedding)}")
print(f"Example of how the embedding model looks like : {sample_embedding}")

Length of the sample_embedding768
Example of how the embedding model looks like : [-0.00018424908921588212, -0.048676688224077225, -0.007528265472501516, -0.021114995703101158, -0.05875687673687935, 0.0468016117811203, -0.023554984480142593, 0.014673741534352303, -0.029078839346766472, 0.0007294445531442761, 0.05068308487534523, 0.03487849235534668, 0.03059239871799946, 0.05017194151878357, 0.010220751166343689, -0.051523879170417786, -0.010246329940855503, 0.002802252070978284, -0.028414595872163773, -0.018820665776729584, -0.05711652711033821, -0.03917144984006882, -0.04345544055104256, 0.03124922700226307, -0.0036828150041401386, -0.0718306228518486, -0.017494220286607742, 0.005189451389014721, -0.026151925325393677, -0.040575239807367325, 0.06681358814239502, -0.0149156479164958, -0.044715698808431625, 0.05305442586541176, 1.9622968920884887e-06, -0.03022927977144718, -0.006832591723650694, 0.004529521334916353, -0.05685825273394585, 0.0005273742717690766, 0.009842011146247387, 0.0

In [112]:
def vector_embedding():
  store = FAISS.from_documents(text_chunks,embeddings)
  embedding = store.as_retriever()
  return embedding
vector_database = vector_embedding()

In [113]:
quiz = "what is the need to cure disease"
result = vector_database.get_relevant_documents(quiz)
#print(f"The result is {result}")
for doc in result:
  print(f"The Result is {doc.page_content}")
  break

The Result is patient will be motivated to a change their lifestyle. 
Note: 
Education that focuses on these identified problems is more likely to be successful than a generic approach toward the condition/medicine. 
 
Education points to consider  
» Focus on the positive aspects of therapy whilst being encouraging 
regarding the impact of the negative aspects and offer support to deal with 
them if they occur. 
» Provide realistic expectations regarding : 
 normal progression of the illness - especially important in those 
diseases where therapy merely controls the progression and those 
that are asymptomatic. 
 the improvement that therapy and non-drug treatment can add to the 
quality of life. 
» Establish therapeutic goals and discuss them openly with the patient. » Any action to be taken with loss of control or when side effects develop. 
» In conditions that are asymptomatic or where symptoms have been


In [114]:
prompt_template = """
Hello mister AI by the way My name is Dineshkumar and You are the AI, your name is BrainWave okayy then if user asks the question you need to give the related answer only clearly check the answer whether it is correct or not
if it's not correct just inform BrainWave AI doesn't know about this and don't make up an answer
<context>
{context}
<query>
{query}
"""

In [115]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate(template = prompt_template , input_variables=["context","query"])

In [116]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm = llm,
            chain_type="stuff",
            retriever=vector_database,
            chain_type_kwargs={"prompt":prompt})

In [117]:
user_input = input("Input Prompt: ")
result = qa_chain({"context": "Provide relevant context here", "query": user_input})
print(result["result"])

Input Prompt: what is ur


ValueError: Missing some input keys: {'query'}

In [ ]:
user_input = input("Input Prompt")
result = qa_chain({"query":user_input})
print(result["result"])

In [ ]:
from langchain_core.tools import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"Medical_Book","Search for Information about the cure of Disease or General Problem of Pain from the Medical Book")

In [ ]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
arxiv_api = ArxivAPIWrapper()
arxiv = ArxivQueryRun(api_wrapper=arxiv_api)

In [ ]:
tools = [wiki,arxiv,retriever_tool]
tools

In [ ]:
prompt = ChatPromptTemplate.from_template("""
You are a helpful assistant. Your work is to act as a doctor here. If a user asks you any question regarding pain or health issues, you need to provide them with medicine and assist them with instructions on how to take it (e.g., morning 1, evening 1, night 1). You can retrieve information from the PDF, research papers, or even from the web.
Don't provide anything further from this

{agent_scratchpad}
""")

In [ ]:
from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [ ]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

In [ ]:
agent_executor.invoke({"input":"Tell me about AI"})